# 🤗 Transformers Episodio 3 - Fine tuning en la nube

[twitch.tv/dataista0](http://twitch.tv/dataista0)


# [Fine-tuning a pretrained model](https://huggingface.co/transformers/training.html#fine-tuning-a-pretrained-model)

* [datasets repo](https://github.com/huggingface/datasets)
* [Datasets doc](https://huggingface.co/docs/datasets/)

# Nos vinimos a la nube, con una Nvidia Tesla T4

* [GPUS en Google Cloud](https://cloud.google.com/compute/docs/gpus)
* [Crear instancia de Google Cloud con GPU attacheada](https://cloud.google.com/compute/docs/gpus/create-vm-with-gpus)

In [3]:
import datasets
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer


#model_name = "distilbert-base-cased"

raw_datasets = datasets.load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))
#full_train_dataset = tokenized_datasets["train"]
#full_eval_dataset = tokenized_datasets["test"]


training_args = TrainingArguments("/home/dataista/git/twitch-streams/data/")

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

Reusing dataset imdb (/home/dataista/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [4]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=39, training_loss=0.6340075761843951, metrics={'train_runtime': 29.1179, 'train_samples_per_second': 10.303, 'train_steps_per_second': 1.339, 'total_flos': 99820164096000.0, 'train_loss': 0.6340075761843951, 'epoch': 3.0})

In [5]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["test"]
)

In [6]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

### BERT consume 10GB de memoria de la GPU y tarda 2hs con los 25K samples del train

In [7]:
len(tokenized_datasets["train"])

25000

In [1]:
import datasets
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer


model_name = "distilbert-base-cased"

raw_datasets = datasets.load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]


training_args = TrainingArguments("/home/dataista/git/twitch-streams/data/")

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


Reusing dataset imdb (/home/dataista/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bia

In [2]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)

In [3]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

### DistilBERT consume 6GB de memoria de la GPU y tarda 1hs con los 25K samples del train

Vamos a agrandar los batch sizes de DistilBERT

In [1]:
import numpy as np
import datasets
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

metric = datasets.load_metric("accuracy")

def get_tokenized_dataset(tokenizer):
    raw_datasets = datasets.load_dataset("imdb")
    
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)    
    
    return tokenized_datasets

def get_tokenizer_and_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    return tokenizer, model

def get_trainer(model, ds_train, ds_eval, args=None):
    if args is None:
        args = TrainingArguments("/home/dataista/git/twitch-streams/data/")
    trainer = Trainer(model=model, args=args, train_dataset=ds_train, eval_dataset=ds_eval)
    return trainer


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
model_name = "distilbert-base-cased"
tokenizer, model = get_tokenizer_and_model(model_name)
ds = get_tokenized_dataset(tokenizer)

small_ds = {
    'train': ds['train'].shuffle(seed=42).select(range(1000)),
    'test': ds['test'].shuffle(seed=42).select(range(1000))
}

args = TrainingArguments("/home/dataista/git/twitch-streams/data/", 
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

In [11]:
trainer = get_trainer(model, ds['train'], ds['test'], args)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bia

Loading cached processed dataset at /home/dataista/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-3a7565679ee901b8.arrow


In [12]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [2]:
model_name = "distilbert-base-cased"
tokenizer, model = get_tokenizer_and_model(model_name)
ds = get_tokenized_dataset(tokenizer)

args = TrainingArguments("/home/dataista/git/twitch-streams/data/", 
                         per_device_train_batch_size=32,
                         per_device_eval_batch_size=32)

trainer = get_trainer(model, ds['train'], ds['test'], args)

try:
    trainer.train()
except Exception as e:
    print(e)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier

Step,Training Loss


KeyboardInterrupt: 

### Sigue tardando una hora con batch size de 32

In [4]:
model_name = "distilbert-base-cased"
tokenizer, model = get_tokenizer_and_model(model_name)
ds = get_tokenized_dataset(tokenizer)

args = TrainingArguments("/home/dataista/git/twitch-streams/data/", 
                         per_device_train_batch_size=32,
                         per_device_eval_batch_size=32)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier

In [7]:
trainer = get_trainer(model, small_ds['train'], small_ds['test'])
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=0.31834248860677083, metrics={'train_runtime': 151.5374, 'train_samples_per_second': 19.797, 'train_steps_per_second': 2.475, 'total_flos': 606256515072000.0, 'train_loss': 0.31834248860677083, 'epoch': 3.0})

In [11]:
args = TrainingArguments("/home/dataista/git/twitch-streams/data/", 
                         per_device_train_batch_size=32,
                         per_device_eval_batch_size=32,
                         evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_ds['train'],
    eval_dataset=small_ds['test'],
    compute_metrics=compute_metrics,
    
)
trainer.evaluate()

{'eval_loss': 0.46108320355415344,
 'eval_accuracy': 0.875,
 'eval_runtime': 19.1594,
 'eval_samples_per_second': 52.194,
 'eval_steps_per_second': 1.67}

In [4]:
args = TrainingArguments("/home/dataista/git/twitch-streams/data/", 
                         per_device_train_batch_size=32,
                         per_device_eval_batch_size=32,
                         evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_ds['train'],
    eval_dataset=small_ds['test'],
    compute_metrics=compute_metrics,
    
)

In [5]:
trainer.evaluate()

{'eval_loss': 0.6938373446464539,
 'eval_accuracy': 0.511,
 'eval_runtime': 17.2968,
 'eval_samples_per_second': 57.814,
 'eval_steps_per_second': 1.85}

In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.510324,0.788000
2,No log,0.319072,0.869000
3,No log,0.374371,0.865000


TrainOutput(global_step=96, training_loss=0.3379857540130615, metrics={'train_runtime': 207.3587, 'train_samples_per_second': 14.468, 'train_steps_per_second': 0.463, 'total_flos': 606256515072000.0, 'train_loss': 0.3379857540130615, 'epoch': 3.0})

In [17]:
import torch
del model
del trainer
torch.cuda.empty_cache()

In [21]:
model_name = "bert-base-cased"
tokenizer, model = get_tokenizer_and_model(model_name)
ds = get_tokenized_dataset(tokenizer)

args = TrainingArguments("/home/dataista/git/twitch-streams/data/", 
                         per_device_train_batch_size=8,
                         per_device_eval_batch_size=8,
                         num_train_epochs=2,
                        evaluation_strategy="epoch")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [19]:
small_ds = {
    'train': ds['train'].shuffle(seed=42).select(range(1000)),
    'test': ds['test'].shuffle(seed=42).select(range(1000))
}

Loading cached shuffled indices for dataset at /home/dataista/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-38de2308bec84f8b.arrow
Loading cached shuffled indices for dataset at /home/dataista/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-f9f3680505d213ba.arrow


In [22]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_ds['train'],
    eval_dataset=small_ds['test'],
    compute_metrics=compute_metrics   
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.471688,0.816000
2,No log,0.458769,0.855000


TrainOutput(global_step=250, training_loss=0.4430783081054688, metrics={'train_runtime': 272.3498, 'train_samples_per_second': 7.343, 'train_steps_per_second': 0.918, 'total_flos': 665467760640000.0, 'train_loss': 0.4430783081054688, 'epoch': 2.0})

In [37]:
ds_pred = ds['test'].shuffle().select(range(20))

Loading cached shuffled indices for dataset at /home/dataista/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-2ee7e3a315ca49bf.arrow


In [43]:
r = trainer.predict(ds_pred)

In [44]:
r

PredictionOutput(predictions=array([[-0.87757206,  0.774027  ],
       [ 2.4134557 , -2.3183644 ],
       [ 2.4195182 , -2.3638883 ],
       [ 2.1845455 , -2.1224418 ],
       [ 2.3230815 , -2.2390404 ],
       [-2.032649  ,  2.0326538 ],
       [-1.849143  ,  1.6348507 ],
       [-2.3554788 ,  2.270405  ],
       [-2.026522  ,  1.8861613 ],
       [-2.2003798 ,  2.1160097 ],
       [ 2.37901   , -2.2737923 ],
       [-2.3000994 ,  2.2109437 ],
       [-1.3952745 ,  1.020727  ],
       [-2.3666618 ,  2.2944164 ],
       [-2.068248  ,  1.9278665 ],
       [-2.2128253 ,  2.1633906 ],
       [-0.16507863,  0.19253802],
       [ 2.3452382 , -2.289456  ],
       [-2.309814  ,  2.1994135 ],
       [ 2.3103797 , -2.2582166 ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0]), metrics={'test_loss': 0.5447111129760742, 'test_accuracy': 0.85, 'test_runtime': 0.7202, 'test_samples_per_second': 27.771, 'test_steps_per_second': 4.166})

### OK, Bert más o menos funciona, vamos a probar todo con un dataset no nativo de la librería

In [48]:
!kaggle competitions download -c nlp-getting-started

  0%|                                                | 0.00/593k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 593k/593k [00:00<00:00, 89.7MB/s]


In [49]:
!unzip nlp-getting-started.zip -d data/nlp-getting-started/

Archive:  nlp-getting-started.zip
  inflating: data/nlp-getting-started/sample_submission.csv  
  inflating: data/nlp-getting-started/test.csv  
  inflating: data/nlp-getting-started/train.csv  


In [51]:
import pandas as pd
data = pd.read_csv('data/nlp-getting-started//train.csv')

In [53]:
df_train = data[['id', 'text', 'target']]

In [54]:
df_train

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


# Nos vamos a otra notebook